## 折衷查表:
### 用 Single Error Correction 解 Double Error Correction (A 取Double_AWE的 A)

#### 1.取 A的bits $\approx$ N的bits $\Rightarrow$ A = 3349 (12bits), N的範圍: 0 ~ 4095 (12bits)

### Define function
* (Not including 2's complement range)

In [2]:
import math
import numpy as np


def decimal_to_binary(decimal_number):
    if decimal_number == 0:
        return "0"
    # 處理正數：直接轉換
    if decimal_number > 0:
        return bin(decimal_number)[2:]  # 移除 "0b" 前綴
       # 處理負數：自動計算所需位元數（2's complement）
    abs_value = abs(decimal_number)
    bits = abs_value.bit_length() + 1  # 負數多一個符號位元
    two_complement_value = (1 << bits) + decimal_number
    
    return bin(two_complement_value)[2:]  # 移除 "0b" 前綴

def bits_needed(number):
    if number < 0:
        raise ValueError("2's complement is not available")  # 如果輸入負數，拋出錯誤
    if number == 0:
        return 1  # 0 需要 1 個位元
    
    # 計算位元數
    bits = math.floor(math.log2(number)) + 1
    return bits


def bits_range(bits, signed=False):     ## 可選擇設定 有號數 或 無號數
    if bits <= 0:
        return "位元數必須為正整數！"
    
    if signed:
        # 二補數範圍: -2^(bits-1) 到 2^(bits-1)-1
        min_value = -(2**(bits - 1))
        max_value = 2**(bits - 1) - 1
    else:
        # 無符號範圍: 0 到 2^bits - 1
        min_value = 0
        max_value = 2**bits - 1

    return min_value, max_value

print("一個2補數所需要的位元數",bits_needed(120))
print("一bits數可表示的2補數的範圍",bits_range(bits_needed(120)))
print("一個2補數表達為二進制為:",decimal_to_binary(-8))

一個2補數所需要的位元數 7
一bits數可表示的2補數的範圍 (0, 127)
一個2補數表達為二進制為: 11000


### 查找表範例

In [3]:
def create_mapping(source_list, address_map):
    """
    建立值到位址和位址到值的雙向映射。
    若 address_map 中某位址沒有對應值，則該位址的值為 0。
    
    參數:
    source_list (list): 原始資料列表。
    address_map (list): 位址映射列表。
    
    回傳:
    tuple: (值到位址的映射, 位址到值的映射)
    """
    # 初始化值到位址的映射
    value_to_address = {value: addr for value, addr in zip(source_list, address_map)}
    
    # 初始化位址到值的映射，未指定的位址預設為 0
    max_address = max(address_map)  # 確保涵蓋所有位址
    address_to_value = {addr: 0 for addr in range(max_address + 1)}  # 預設為 0
    for value, addr in zip(source_list, address_map):
        address_to_value[addr] = value  # 更新有指定值的位址

    return value_to_address, address_to_value

# 測試數據
source_list = [10, 20, 30]
address_map = [2, 0, 4]

# 建立映射
value_to_address, address_to_value = create_mapping(source_list, address_map)

# 顯示結果
print("值到位址的映射:", value_to_address)
print("位址到值的映射:", address_to_value)


# 測試數據
source_list = [10, 20, 30, 40]
address_map = [2, 0, 3, 1]

# 建立映射
value_to_address, address_to_value = create_mapping(source_list, address_map)

# 測試查詢
print("值到位址:", value_to_address)  # 查詢 10 -> 2, 20 -> 0
print("位址到值:", address_to_value)  # 查詢 2 -> 10, 0 -> 20

# 範例查詢
value = 10
address = 2
print(f"輸入值 {value}, 對應位址: {value_to_address[value]}")
print(f"輸入位址 {address}, 對應值: {address_to_value[address]}")

值到位址的映射: {10: 2, 20: 0, 30: 4}
位址到值的映射: {0: 20, 1: 0, 2: 10, 3: 0, 4: 30}
值到位址: {10: 2, 20: 0, 30: 3, 40: 1}
位址到值: {0: 20, 1: 40, 2: 10, 3: 30}
輸入值 10, 對應位址: 2
輸入位址 2, 對應值: 10


## 以12bits為例:

In [4]:
### 輸入數值為data, ANcode的A為AN
### 乘完A後的數值為 input
data = 4095     ## 0~4095
AN = 3349
input = data*AN
bits = bits_needed(input)
print("AN資料的位元數:", bits)

AN資料的位元數: 24


### 產生1-bit AWE (Arithmetic-Weight Error)

In [5]:
AWE_1=[]
e1=0     ## 錯誤數量
e1_data=[]
for i in range(0, bits):
    AWE_1.insert(e1 , input + 2**i)
    e1_data.insert(e1, 2**i)
    e1 = e1 + 1
    AWE_1.insert(e1 , input - 2**i)
    e1_data.insert(e1, -2**i)
    e1 = e1 + 1

print(bits)
print(input)
print("產生1-bit AWE:",AWE_1)
print("算術錯誤的內容:", e1_data)
print(e1)
print(np.size(AWE_1))

24
13714155
產生1-bit AWE: [13714156, 13714154, 13714157, 13714153, 13714159, 13714151, 13714163, 13714147, 13714171, 13714139, 13714187, 13714123, 13714219, 13714091, 13714283, 13714027, 13714411, 13713899, 13714667, 13713643, 13715179, 13713131, 13716203, 13712107, 13718251, 13710059, 13722347, 13705963, 13730539, 13697771, 13746923, 13681387, 13779691, 13648619, 13845227, 13583083, 13976299, 13452011, 14238443, 13189867, 14762731, 12665579, 15811307, 11617003, 17908459, 9519851, 22102763, 5325547]
算術錯誤的內容: [1, -1, 2, -2, 4, -4, 8, -8, 16, -16, 32, -32, 64, -64, 128, -128, 256, -256, 512, -512, 1024, -1024, 2048, -2048, 4096, -4096, 8192, -8192, 16384, -16384, 32768, -32768, 65536, -65536, 131072, -131072, 262144, -262144, 524288, -524288, 1048576, -1048576, 2097152, -2097152, 4194304, -4194304, 8388608, -8388608]
48
48


### 產生SEC的模數:

In [16]:

def custom_mod(a, b):
    if b == 0:
        return "錯誤：除數不能為零！"
    # 計算商和餘數，修正餘數計算邏輯
    quotient = a // b
    remainder = a - b * quotient
    # 修正負餘數的情況
    if remainder > 0 and a < 0:
        remainder -= abs(b)
        quotient += 1
    return remainder

SEC_R = []
for m in range(0,e1):
    mod = custom_mod(AWE_1[m], AN)
    SEC_R.append(mod)

print("SEC的模數:\n",SEC_R)
print("SEC的大小:",np.size(SEC_R))

SEC的模數:
 [1, 3348, 2, 3347, 4, 3345, 8, 3341, 16, 3333, 32, 3317, 64, 3285, 128, 3221, 256, 3093, 512, 2837, 1024, 2325, 2048, 1301, 747, 2602, 1494, 1855, 2988, 361, 2627, 722, 1905, 1444, 461, 2888, 922, 2427, 1844, 1505, 339, 3010, 678, 2671, 1356, 1993, 2712, 637]
SEC的大小: 48


## 建立只能做SEC的LUT

In [7]:
AWE_to_SEC_R, SEC_R_to_AWE = create_mapping(e1_data, SEC_R)

### 產生2-bit AWE (先固定某位元)

In [24]:
AWE_2=[]
e2=0     ## 錯誤數量
p=0      ## 錯誤位置
e2_data=[]
N = 12
SEC_value = -2**N     ## 先取某一固定位元,AWE為: (+) or (-) 2^N
print("對應的餘數:",AWE_to_SEC_R[SEC_value])
for j in range(0, bits):
    e2_data.insert(e2 , (input + SEC_value + 2**j))
    AWE_2.insert(e2 , (SEC_value + 2**j))        
    print(f"第 {e2} 筆錯誤資料: {e2_data[e2]}")
    print(f"第 {e2} 筆算術錯誤: {AWE_2[e2]}")
    e2 = e2 + 1
    e2_data.insert(e2 , (input + SEC_value - 2**j))
    AWE_2.insert(e2 , (SEC_value - 2**j))        
    print(f"第 {e2} 筆錯誤資料: {e2_data[e2]}")
    print(f"第 {e2} 筆算術錯誤: {AWE_2[e2]}")
    e2 = e2 + 1
    p = p + 1
####################################################
del AWE_2[(2*N)+1]
del AWE_2[2*N]
del e2_data[(2*N)+1]
del e2_data[2*N]
####################################################
p = p -1
e2 = e2 - 2
print(e2)
print(p)
print("產生2-bits AWE:",AWE_2)
print("產生2-bits e2_data:",e2_data)

對應的餘數: 2602
第 0 筆錯誤資料: 13710060
第 0 筆算術錯誤: -4095
第 1 筆錯誤資料: 13710058
第 1 筆算術錯誤: -4097
第 2 筆錯誤資料: 13710061
第 2 筆算術錯誤: -4094
第 3 筆錯誤資料: 13710057
第 3 筆算術錯誤: -4098
第 4 筆錯誤資料: 13710063
第 4 筆算術錯誤: -4092
第 5 筆錯誤資料: 13710055
第 5 筆算術錯誤: -4100
第 6 筆錯誤資料: 13710067
第 6 筆算術錯誤: -4088
第 7 筆錯誤資料: 13710051
第 7 筆算術錯誤: -4104
第 8 筆錯誤資料: 13710075
第 8 筆算術錯誤: -4080
第 9 筆錯誤資料: 13710043
第 9 筆算術錯誤: -4112
第 10 筆錯誤資料: 13710091
第 10 筆算術錯誤: -4064
第 11 筆錯誤資料: 13710027
第 11 筆算術錯誤: -4128
第 12 筆錯誤資料: 13710123
第 12 筆算術錯誤: -4032
第 13 筆錯誤資料: 13709995
第 13 筆算術錯誤: -4160
第 14 筆錯誤資料: 13710187
第 14 筆算術錯誤: -3968
第 15 筆錯誤資料: 13709931
第 15 筆算術錯誤: -4224
第 16 筆錯誤資料: 13710315
第 16 筆算術錯誤: -3840
第 17 筆錯誤資料: 13709803
第 17 筆算術錯誤: -4352
第 18 筆錯誤資料: 13710571
第 18 筆算術錯誤: -3584
第 19 筆錯誤資料: 13709547
第 19 筆算術錯誤: -4608
第 20 筆錯誤資料: 13711083
第 20 筆算術錯誤: -3072
第 21 筆錯誤資料: 13709035
第 21 筆算術錯誤: -5120
第 22 筆錯誤資料: 13712107
第 22 筆算術錯誤: -2048
第 23 筆錯誤資料: 13708011
第 23 筆算術錯誤: -6144
第 24 筆錯誤資料: 13714155
第 24 筆算術錯誤: 0
第 25 筆錯誤資料: 13705963
第 25 筆算術錯誤: -819

### 產生DEC模數: (先固定某一位元為例)

In [25]:
DEC_R = []
for m in range(0,e2):
    mod = custom_mod(e2_data[m], AN)
    DEC_R.append(mod)

print("DEC_R的模數:\n",DEC_R)
print(np.size(DEC_R))


DEC_R的模數:
 [2603, 2601, 2604, 2600, 2606, 2598, 2610, 2594, 2618, 2586, 2634, 2570, 2666, 2538, 2730, 2474, 2858, 2346, 3114, 2090, 277, 1578, 1301, 554, 747, 1108, 2241, 2963, 1880, 3324, 1158, 697, 3063, 2141, 175, 1680, 1097, 758, 2941, 2263, 3280, 1924, 609, 1246, 1965, 3239]
46


### 尋找SEC和DEC關係:

In [26]:
SEC_R2 = []
position = 0
AN = 3349
for i in range(len(DEC_R)):
    SEC_R2.insert(position, (DEC_R[i] - AWE_to_SEC_R[SEC_value]))  ##  R2 = R - R1
    position = position+1
    
for i in range(len(SEC_R2)):
    if SEC_R2[i] < 0:
        SEC_R2[i] = AN + SEC_R2[i]    ## 若值小於0, AN+負值
   
print(SEC_R2) 
print(len(SEC_R2))


[1, 3348, 2, 3347, 4, 3345, 8, 3341, 16, 3333, 32, 3317, 64, 3285, 128, 3221, 256, 3093, 512, 2837, 1024, 2325, 2048, 1301, 1494, 1855, 2988, 361, 2627, 722, 1905, 1444, 461, 2888, 922, 2427, 1844, 1505, 339, 3010, 678, 2671, 1356, 1993, 2712, 637]
46


In [28]:
SEC_AWE2 = [] 
position = 0
for A in range(len(SEC_R2)):
    SEC_AWE2.insert(position,(SEC_R_to_AWE[SEC_R2[A]]))
    position = position + 1

print(SEC_AWE2)

[1, -1, 2, -2, 4, -4, 8, -8, 16, -16, 32, -32, 64, -64, 128, -128, 256, -256, 512, -512, 1024, -1024, 2048, -2048, 8192, -8192, 16384, -16384, 32768, -32768, 65536, -65536, 131072, -131072, 262144, -262144, 524288, -524288, 1048576, -1048576, 2097152, -2097152, 4194304, -4194304, 8388608, -8388608]


### Trade-off algorithm:(以下為大致流程)

![Trade-off演算法](./圖片/Trade-off%20algorithm.png )

In [29]:
## 把重複的 AWE 去除
def remove_duplicates(input_list):
    unique_list = []
    for number in input_list:
        if number not in unique_list:
            unique_list.append(number)
    return unique_list

In [31]:
###給定一個DEC_R, 找SEC_R1及SEC_R2
import math

print("產生2-bits AWE:\n",AWE_2)
print("Double AWE的餘數:\n",DEC_R)
AWE1 = 0 ; AWE2 = 0; R1 = 0; R2 = 0
possible_AWE = []; possible_AWE1 = []; possible_AWE2 = []
for R in DEC_R:
    for errors in range(0,bits):
        for o in range(2):                         ### 0:+, 1:-
            if o == 0:
                AWE1 = +2**errors
            else:
                AWE1 = -2**errors
            R1 = AWE_to_SEC_R[AWE1]
            R2 = R - R1
            if R2 < 0: 
                R2 = AN + R2
            AWE2 = SEC_R_to_AWE[R2]               ### 若找不到對應的AWE2,
            if AWE2 != 0:                         ### AWE2定為0(函數內容已定義)  
                possible_AWE.append(AWE1+AWE2)
                #possible_AWE1.append(AWE1)
                #possible_AWE2.append(AWE1)
            
possible_AWE = remove_duplicates(possible_AWE)    ### 把重複的AWE去除     
print("用折衷法 (Trade-off) 解出來的 Double AWE:\n",possible_AWE)
print("是否和產生2-bits AWE相同:",possible_AWE == AWE_2)

產生2-bits AWE:
 [-4095, -4097, -4094, -4098, -4092, -4100, -4088, -4104, -4080, -4112, -4064, -4128, -4032, -4160, -3968, -4224, -3840, -4352, -3584, -4608, -3072, -5120, -2048, -6144, 4096, -12288, 12288, -20480, 28672, -36864, 61440, -69632, 126976, -135168, 258048, -266240, 520192, -528384, 1044480, -1052672, 2093056, -2101248, 4190208, -4198400, 8384512, -8392704]
Double AWE的餘數:
 [2603, 2601, 2604, 2600, 2606, 2598, 2610, 2594, 2618, 2586, 2634, 2570, 2666, 2538, 2730, 2474, 2858, 2346, 3114, 2090, 277, 1578, 1301, 554, 747, 1108, 2241, 2963, 1880, 3324, 1158, 697, 3063, 2141, 175, 1680, 1097, 758, 2941, 2263, 3280, 1924, 609, 1246, 1965, 3239]
用折衷法 (Trade-off) 解出來的 Double AWE:
 [-4095, -4097, -4094, -4098, -4092, -4100, -4088, -4104, -4080, -4112, -4064, -4128, -4032, -4160, -3968, -4224, -3840, -4352, -3584, -4608, -3072, -5120, -2048, -6144, 4096, -12288, 12288, -20480, 28672, -36864, 61440, -69632, 126976, -135168, 258048, -266240, 520192, -528384, 1044480, -1052672, 2093056, -2